In [56]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.preprocessing import normalize

In [66]:
train_data = pd.read_csv('./汽车领域文本规则泛化性增强挑战赛公开数据/train.csv')
train_data['规则id'] = train_data['规则id'].astype(str)

test_data = pd.read_csv('./汽车领域文本规则泛化性增强挑战赛公开数据/test_a.csv')

rules = pd.read_excel('./汽车领域文本规则泛化性增强挑战赛公开数据/规则数据.xlsx')
rules['规则id'] = rules['规则id'].astype(str)

train_data = pd.merge(train_data, rules, left_on='规则id', right_on='规则id', how='left')
train_data = train_data.sample(frac=1.0)

In [99]:
train_examples = []
for row in train_data.iterrows():
    neg_rules = rules['规则表达式'].sample(5).values
    pos_rule = row[1]['规则表达式']

    for text in neg_rules:
        if text == row[1]['规则表达式']:
            continue
        train_examples.append(InputExample(texts=[row[1].text, text], label=0.0))

    if pos_rule is not np.nan:
        train_examples.append(InputExample(texts=[row[1].text, pos_rule], label=1.0))

In [100]:
len(train_examples)

30775

In [101]:
from sentence_transformers import SentenceTransformer, models, losses, InputExample, evaluation
from torch import nn
from torch.utils.data import DataLoader

# https://huggingface.co/moka-ai/m3e-small
# model = SentenceTransformer('moka-ai/m3e-small')

word_embedding_model = models.Transformer('/home/lyz/下载/m3e-small/', max_seq_length=80)

tokens = ["{open}", "{close}", "{set_up}", "{set_down}", "{set}", "{set_min}", "{set_max}", "{NUM}"]
word_embedding_model.tokenizer.add_tokens(tokens, special_tokens=True)
word_embedding_model.auto_model.resize_token_embeddings(len(word_embedding_model.tokenizer))

pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=256, activation_function=nn.Tanh())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])

model = SentenceTransformer('/home/lyz/下载/m3e-small/')
train_dataloader = DataLoader(train_examples[:], shuffle=True, batch_size=16)
# val_dataloader = DataLoader(train_examples[-6000:], shuffle=True, batch_size=16)

No sentence-transformers model found with name /home/lyz/下载/m3e-small/. Creating a new one with MEAN pooling.


In [102]:
train_loss = losses.CosineSimilarityLoss(model)
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=5, warmup_steps=10)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1924 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1924 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1924 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1924 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1924 [00:00<?, ?it/s]

In [103]:
train_data

,text,规则id,规则表达式,语义参考
176,打开空调打开冷空调,None,NaN,NaN
2255,空调关低点,22,空调{set_down}\n空调(温度){set_down}\n温度{set_down},"{""temperature"":""minus""}@set@aircontrol"
2016,空调最冷,14,空调(温度){set_min}\n空调{set_min}\n温度{set}{set_min}...,"{""temperature"":""min""}@set@aircontrol"
5106,请设置到导航全屏模式,540,({open}|{set})[到]导航全屏[模式],"{""insType"":""EXIT_FULL_SCREEN""}@mapu"
4562,嗯打开音乐,503,{open}(音乐|歌),"{""instype"":""open""}@instruction@musicx"
...,...,...,...,...
4850,帮我设置自定义唤醒词,529,(设置|修改)自定义唤醒词,"{""insType"":""OPEN_CUSTOM_WAKEUP_WORD""}@cmd"
2461,自动空调,30,空调{set}自动模式\n空调自动\n空调{set}自动\n{open}自动模式,"{""mode"":""自动""}@set@aircontrol"
2466,空调调成自动,30,空调{set}自动模式\n空调自动\n空调{set}自动\n{open}自动模式,"{""mode"":""自动""}@set@aircontrol"
411,打开空调制冷一档,None,NaN,NaN


In [110]:
test_feat = model.encode(list(test_data['text']))
test_feat = normalize(test_feat)

In [111]:
rules_feat = model.encode(list(rules['规则表达式']))
rules_feat = normalize(rules_feat)

In [119]:
dist = 1 - cdist(test_feat, rules_feat, metric='cosine')

In [129]:
test_rule = []
for idx in range(len(dist)):
    if dist[idx].max() < 0.8:
        test_rule.append(None)
    else:
        test_rule.append(rules['规则id'].iloc[dist[idx].argmax()])

In [131]:
test_data['规则id'] = test_rule

In [133]:
test_data.to_csv('submit.csv', index=None)